We hope to download publication data and construct affiliation data for all the graduate student and PIs.

The first step is to identify the scopus authorid for each person. 
To do this we need to download a list of publications (a sample of these individual's publications) based on the pmids Chris shared with us, and use these publication sample to identify the authorid.
Then we use authorid to doanload all their publications. 

We have finished the graduate student data. Now we focus on Principal investigators (PIs)

Step 1: use the list of pmid (from "a list of PI pmid.dta") to download all the publication author information, create a table (named "publication sample") including 
auid, indexed_name, surname, given_name, eid, pmid, auth_posn.

Step 2: merge the data with "a list of PI pmid.dta" using pmid and authorposition (only keep the merged records), create a table with variables:

eid, auid, indexed_name, surname, given_name, cliuid.

Step 3: create a list of auid from above (named PI authorid.dta"), and use the author retrieval function (https://scopus.readthedocs.io/en/stable/reference/scopus.AuthorRetrieval.html) 
from Scopus to pull all the publications for each indivudals. Two functions can be used  to generate a list of eid ("get_document_eids").

Then create a table named "PI publication.dta" with variables:

eid, auid, auth_posn, pmid

Step 4. Use all the eids to download all the publication information as we did before (as below). 

However, this time we need more information about the publication, meaning Table 1 has more variables. On top of the variables we used last time, we need to download the following information.

(1) publicationName (or  title), aggregationType, journalname (or journal ), coverDate, volume, issueIdentifier, pageRange (or pages), doi, publicationyear 

(2) idxterms, subject_areas, authkeywords

(3) funding informaiton: funding,funding_text, chemicals, sequencebank

(4) If possible: also download issn.

In [9]:
import pandas as pd

df1 = pd.read_stata("a list of PI pmid.dta")
df1 = df1.rename(index=str, columns={'pmid_authposn': 'auth_posn'})
df1

,cliuid_2,pmid,auth_posn
0,A4801028,3574384,2
1,A4801028,13291872,3
2,A4801028,12983668,2
3,A4801028,4547285,2
4,A4801028,7583245,4
5,A4901255,14834152,3
6,A4901255,16590607,4
7,A4901255,16590214,2
8,A4901255,17736111,1
9,A4901255,16589608,3


In [2]:
from scopus import AbstractRetrieval

pmid = df1['pmid']

ab = AbstractRetrieval(pmid[0], view='FULL')

df2 = pd.DataFrame(ab.authors)
df2['pmid'] = pmid[0] # add pmid column

# Add author position
for i in range(len(df2)):
    df2.loc[i,'auth_posn'] = i+1
    
df2 = df2.drop('affiliation',axis=1)
df2['auth_posn'] = df2['auth_posn'].astype(int)
df2.to_csv('PI_publication_sample.csv',index=False)
df2

,auid,indexed_name,surname,given_name,pmid,auth_posn
0,16482993600,Geiger G.,Geiger,Gad,3574384,1
1,57207897457,Lettvin J.Y.,Lettvin,Jerome Y.,3574384,2


In [3]:
# Step 1
for i in range(1,len(pmid)):
    try:
        ab = AbstractRetrieval(pmid[i], view='FULL')
        df2 = pd.DataFrame(ab.authors)
        df2['pmid'] = pmid[i]
        # Add author position
        for j in range(len(df2)):
            df2.loc[j,'auth_posn'] = j+1
        df2['auth_posn'] = df2['auth_posn'].astype(int)
        df2 = df2.drop('affiliation',axis=1)
        df2.to_csv('PI_publication_sample.csv', mode='a', header= False, index=False) # Step 1
    except:
        pass

In [10]:
# Step 2
df2 = pd.read_csv('PI_publication_sample.csv')
df3 = pd.merge(df1, df2, how='inner', on=['pmid','auth_posn']).drop_duplicates()
df3.to_csv('PI_authorid.csv',index=False) 
df3

,cliuid_2,pmid,auth_posn,auid,indexed_name,surname,given_name
0,A4801028,3574384,2,57207897457,Lettvin J.Y.,Lettvin,Jerome Y.
1,A4801028,13291872,3,57207897457,LETTVIN J.Y.,LETTVIN,J. Y.
2,A4801028,12983668,2,57207897457,LETTVIN J.Y.,LETTVIN,J. Y.
3,A4801028,4547285,2,57207897457,Lettvin J.Y.,Lettvin,J. Y.
4,A4801028,7583245,4,57207897457,Lettvin J.Y.,Lettvin,Jerome Y.
5,A4901255,14834152,3,7202545622,SCHMITT F.O.,SCHMITT,F. O.
6,A5101509,4551832,1,24292197700,Wall P.D.,Wall,P. D.
7,A5101509,2626605,4,24292197700,Wall P.D.,Wall,Patrick D.
8,A5101509,11486755,1,24292197700,Wall P.D.,Wall,Patrick D.
9,A5101509,14205021,2,24292197700,Wall P.D.,Wall,P. D.


In [13]:
from scopus import AuthorRetrieval

df3 = pd.read_csv('PI_authorid.csv')
df3['eid'] = None
for i in range(len(df3)):
    try:
        au = AuthorRetrieval(df3['auid'][i]) 
        df3.at[i,'eid'] = au.get_document_eids(refresh=False)
    except:
        pass
df3

C:\Users\kd_ch\Anaconda3\lib\site-packages\scopus\author_retrieval.py:92: UserWarning: Profile with ID 55197103800 has been merged and the new ID is 7102270696.  Please update your records manually.  Files have been cached with the old ID.
  warn(text, UserWarning)


,cliuid_2,pmid,auth_posn,auid,indexed_name,surname,given_name,eid
0,A4801028,3574384,2,57207897457,Lettvin J.Y.,Lettvin,Jerome Y.,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0..."
1,A4801028,13291872,3,57207897457,LETTVIN J.Y.,LETTVIN,J. Y.,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0..."
2,A4801028,12983668,2,57207897457,LETTVIN J.Y.,LETTVIN,J. Y.,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0..."
3,A4801028,4547285,2,57207897457,Lettvin J.Y.,Lettvin,J. Y.,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0..."
4,A4801028,7583245,4,57207897457,Lettvin J.Y.,Lettvin,Jerome Y.,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0..."
5,A4901255,14834152,3,7202545622,SCHMITT F.O.,SCHMITT,F. O.,"[2-s2.0-0021944589, 2-s2.0-0021736481, 2-s2.0-..."
6,A5101509,4551832,1,24292197700,Wall P.D.,Wall,P. D.,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2...."
7,A5101509,2626605,4,24292197700,Wall P.D.,Wall,Patrick D.,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2...."
8,A5101509,11486755,1,24292197700,Wall P.D.,Wall,Patrick D.,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2...."
9,A5101509,14205021,2,24292197700,Wall P.D.,Wall,P. D.,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2...."


In [14]:
# Step 3
df4 = df3[['eid','auid','auth_posn','pmid']]
df4.to_csv('PI_publication.csv',index=False) 
df4

,eid,auid,auth_posn,pmid
0,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0...",57207897457,2,3574384
1,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0...",57207897457,3,13291872
2,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0...",57207897457,2,12983668
3,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0...",57207897457,2,4547285
4,"[2-s2.0-85041979579, 2-s2.0-4143128954, 2-s2.0...",57207897457,4,7583245
5,"[2-s2.0-0021944589, 2-s2.0-0021736481, 2-s2.0-...",7202545622,3,14834152
6,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2....",24292197700,1,4551832
7,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2....",24292197700,4,2626605
8,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2....",24292197700,1,11486755
9,"[2-s2.0-84956711522, 2-s2.0-84956506865, 2-s2....",24292197700,2,14205021


### Tables 1-5

In [16]:
import pandas as pd
import ast
from scopus import AbstractRetrieval

df4 = pd.read_csv('PI_publication.csv')
lol = list(df4['eid']) # list of lists
print(lol[0][0]) # bracket
print(ast.literal_eval(lol[0])[0]) # eid

lst = [] # Convert lol to just a single list
for i in range(len(lol)):
    try:
        for item in ast.literal_eval(lol[i]):
            lst.append(item)
    except:
        pass
print(len(lst))
eid = list(set(lst)) # Remove duplicates
len(eid)

[
2-s2.0-85041979579
104578


23322

In [23]:
# Step 4
ab = AbstractRetrieval(eid[1], view='FULL')

table1 = pd.DataFrame(ab.authors)
table1['eid'] = eid[1] # add eid column

# Add author position
for i in range(len(table1)):
    table1.loc[i,'auth_posn'] = i+1

# Add additional columns
table1['publicationName'] = ab.publicationName
table1['aggregationType'] = ab.aggregationType
table1['coverDate'] = ab.coverDate
table1['volume'] = ab.volume
table1['issueIdentifier'] = ab.issueIdentifier
table1['pageRange'] = ab.pageRange
table1['doi'] = ab.doi
table1['PublicationYear'] = ab.get_ris().split('\n')[5][-4:]
table1['idxterms'] = ''
table1['subject_areas'] = ''
table1['authkeywords'] = ''
table1['funding'] = ''
table1['funding_text'] = ab.funding_text
table1['chemicals'] = ''
table1['sequencebank'] = ab.sequencebank
table1['issn'] = ab.issn
for i in range(len(table1)):
    table1.at[i,'idxterms'] = ab.idxterms
    table1.at[i,'subject_areas'] = ab.subject_areas
    table1.at[i,'authkeywords'] = ab.authkeywords
    table1.at[i,'funding'] = ab.funding
    table1.at[i,'chemicals'] = ab.chemicals

# Now we need to split up lists of affiliation ids to have 1 per row.

# Split affiliations in separate rows. First create a dataframe of affiliations
temp = table1.affiliation.apply(pd.Series)

# Merge table1 with temp, and clean up the data.
table1 = table1.merge(temp, left_index = True, right_index = True) \
             .drop(['affiliation'],axis=1) \
             .melt(id_vars = ['auid','eid','indexed_name','surname','given_name','auth_posn',\
                              'publicationName','aggregationType','coverDate','volume','issueIdentifier',\
                              'pageRange','doi','PublicationYear','idxterms','subject_areas','authkeywords',\
                              'funding','funding_text','chemicals','sequencebank','issn'],\
                   value_name = "affiliation_id") \
             .drop(['variable'], axis=1) \
             .dropna(subset=['affiliation_id'])
table1

,auid,eid,indexed_name,surname,given_name,auth_posn,publicationName,aggregationType,coverDate,volume,...,PublicationYear,idxterms,subject_areas,authkeywords,funding,funding_text,chemicals,sequencebank,issn,affiliation_id
0,12799271400,2-s2.0-76549129029,Gresham D.,Gresham,David,1.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269
1,7006974621,2-s2.0-76549129029,Curry B.,Curry,Bo,2.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60024780
2,15049126000,2-s2.0-76549129029,Ward A.,Ward,Alexandra,3.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269
3,55228966500,2-s2.0-76549129029,Gordon D.B.,Gordon,D. Benjamin,4.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60024780
4,7004075986,2-s2.0-76549129029,Brizuela L.,Brizuela,Leonardo,5.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60024780
5,7004989003,2-s2.0-76549129029,Kruglyak L.,Kruglyak,Leonid,6.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269
6,7102711191,2-s2.0-76549129029,Botstein D.,Botstein,David,7.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269
7,12799271400,2-s2.0-76549129029,Gresham D.,Gresham,David,1.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269
9,15049126000,2-s2.0-76549129029,Ward A.,Ward,Alexandra,3.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269
12,7004989003,2-s2.0-76549129029,Kruglyak L.,Kruglyak,Leonid,6.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,2010,None,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269


In [24]:
table2 = pd.DataFrame(ab.authorgroup)
table2['eid'] = eid[1] # add eid column
table2 = table2[['eid','auid','affiliation_id','organization','city_group']] # keep necessary variables
table2

,eid,auid,affiliation_id,organization,city_group
0,2-s2.0-76549129029,12799271400,60003269,"Department of Molecular Biology, Princeton Uni...","Princeton, NJ 08544"
1,2-s2.0-76549129029,15049126000,60003269,"Department of Molecular Biology, Princeton Uni...","Princeton, NJ 08544"
2,2-s2.0-76549129029,7102711191,60003269,"Department of Molecular Biology, Princeton Uni...","Princeton, NJ 08544"
3,2-s2.0-76549129029,12799271400,60003269,Lewis Sigler Institute for Integrative Genomic...,"Princeton, NJ 08544"
4,2-s2.0-76549129029,15049126000,60003269,Lewis Sigler Institute for Integrative Genomic...,"Princeton, NJ 08544"
5,2-s2.0-76549129029,7004989003,60003269,Lewis Sigler Institute for Integrative Genomic...,"Princeton, NJ 08544"
6,2-s2.0-76549129029,7102711191,60003269,Lewis Sigler Institute for Integrative Genomic...,"Princeton, NJ 08544"
7,2-s2.0-76549129029,12799271400,60021784,"Center for Genomics and Systems Biology, Depar...","New York, NY 10003"
8,2-s2.0-76549129029,7006974621,60024780,Agilent Technologies,"Santa Clara, CA 95051"
9,2-s2.0-76549129029,55228966500,60024780,Agilent Technologies,"Santa Clara, CA 95051"


In [25]:
table3 = pd.merge(table1, table2, how='left', on=['auid','eid','affiliation_id'])
table3

,auid,eid,indexed_name,surname,given_name,auth_posn,publicationName,aggregationType,coverDate,volume,...,subject_areas,authkeywords,funding,funding_text,chemicals,sequencebank,issn,affiliation_id,organization,city_group
0,12799271400,2-s2.0-76549129029,Gresham D.,Gresham,David,1.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269,"Department of Molecular Biology, Princeton Uni...","Princeton, NJ 08544"
1,12799271400,2-s2.0-76549129029,Gresham D.,Gresham,David,1.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269,Lewis Sigler Institute for Integrative Genomic...,"Princeton, NJ 08544"
2,7006974621,2-s2.0-76549129029,Curry B.,Curry,Bo,2.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60024780,Agilent Technologies,"Santa Clara, CA 95051"
3,15049126000,2-s2.0-76549129029,Ward A.,Ward,Alexandra,3.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269,"Department of Molecular Biology, Princeton Uni...","Princeton, NJ 08544"
4,15049126000,2-s2.0-76549129029,Ward A.,Ward,Alexandra,3.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269,Lewis Sigler Institute for Integrative Genomic...,"Princeton, NJ 08544"
5,55228966500,2-s2.0-76549129029,Gordon D.B.,Gordon,D. Benjamin,4.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60024780,Agilent Technologies,"Santa Clara, CA 95051"
6,7004075986,2-s2.0-76549129029,Brizuela L.,Brizuela,Leonardo,5.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60024780,Agilent Technologies,"Santa Clara, CA 95051"
7,7004989003,2-s2.0-76549129029,Kruglyak L.,Kruglyak,Leonid,6.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269,Lewis Sigler Institute for Integrative Genomic...,"Princeton, NJ 08544"
8,7004989003,2-s2.0-76549129029,Kruglyak L.,Kruglyak,Leonid,6.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269,Department of Ecology and Evolutionary Biology...,"Princeton, NJ 08544"
9,7004989003,2-s2.0-76549129029,Kruglyak L.,Kruglyak,Leonid,6.0,Proceedings of the National Academy of Science...,Journal,2010-01-26,107,...,"[(Multidisciplinary, MULT, 1000)]","[DNA/DNA hybridization, Melting temperature, P...",None,None,"[(esbd, DNA, 9007-49-2), (nlm, DNA Probes, 0)]",None,00278424 10916490,60003269,"Howard Hughes Medical In

In [26]:
ab = AbstractRetrieval(eid[1], view='FULL')
tab = pd.DataFrame(ab.authorgroup)
aff_ids = []
for item in tab['affiliation_id']:
    if item not in aff_ids:
        aff_ids.append(item)
aff_ids

['60003269', '60021784', '60024780']

In [27]:
from scopus import ContentAffiliationRetrieval

table4 = pd.DataFrame()
table5 = pd.DataFrame()
for item in aff_ids:
    aff = ContentAffiliationRetrieval(item)
    row4 = pd.DataFrame([[item, aff.affiliation_name, aff.sort_name, aff.org_type, aff.postal_code, aff.city, \
                          aff.state, aff.country, aff.org_domain, aff.org_URL]], \
                          columns = ['affiliation_id', 'affiliation_name', 'sort_name', 'org_type', 'postal_code',\
                                     'city', 'state', 'country', 'org_domain', 'org_URL'])
    table4 = table4.append(row4)
    row5 = pd.DataFrame(aff.name_variants)
    row5['affiliation_id'] = item
    table5 = table5.append(row5)
table4

,affiliation_id,affiliation_name,sort_name,org_type,postal_code,city,state,country,org_domain,org_URL
0,60003269,Princeton University,Princeton University,univ,8544,Princeton,NJ,United States,princeton.edu,http://www.princeton.edu/main/
0,60021784,New York University,New York University,univ,10012-1019,New York,NY,United States,nyu.edu,http://www.nyu.edu/
0,60024780,Agilent Technologies,Agilent Technologies,comp,95051,Santa Clara,CA,United States,agilent.com,http://www.agilent.com/about/index.html


In [28]:
table5

,name,doc_count,affiliation_id
0,Princeton University,116752,60003269
1,Princeton Univ,4367,60003269
2,Princeton Univ.,1222,60003269
0,New York University,80397,60021784
1,University Of New York,2793,60021784
2,Nyu,1285,60021784
3,New York University School Of Law,932,60021784
4,New York Univ.,906,60021784
5,New York Univ,830,60021784
6,New York University College Of Medicine,750,60021784


In [29]:
table1.to_csv('PI_table1.csv', index=False)
table2.to_csv('PI_table2.csv', index=False)